# Stream Model From GCP Cloud Storage

In this notebook we will demonstrate how to read tensors using the RunAI Model Streamer from GCP Cloud Storage bucket.

## Prerequisites
* A Linux machine
* A GCP Cloud Storage bucket
* HMAC Credentials for a GCP Service Account that can read and write to the bucket (roles/storage.objectUser)

Install python libraries

In [ ]:
!pip install boto3 runai-model-streamer runai-model-streamer-gcs

## Preparation
Set the following variables with your credentials and bucket:

In [ ]:
gcs_bucket = 'your-bucket-name'
gcs_location = 'desired/path/in/gcs'

### Credentials
Select one of the three supported methods of authentication:

In [ ]:
# 1. External Credentials

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '~/.config/gcloud/application_default_credentials.json'

In [ ]:
# 2. Service Account Credentials

import os
sa_key_file = "~/path/to/sa_key.json"
os.environ['RUNAI_STREAMER_GCS_CREDENTIAL_FILE'] = sa_key_file

We will start by downloading an example `.safetensors` file. Feel free to use your own.

Next thing we would do is upload the `model.safetensors` file to the Cloud Storage bucket:

In [ ]:
from google.cloud import storage
import google.auth

if sa_key_file:
    creds = google.auth.load_credentials_from_file(sa_key_file)
else:
    creds = google.auth.default()

local_file = 'model.safetensors'
gcs_upload_location = f"{gcs_location}/{local_file}"

gcs_client = storage.Client(credentials = creds)
bucket = gcs_client.bucket(gcs_bucket)
blob = bucket.blob(gcs_upload_location)
blob.upload_from_filename(local_file)

## Streaming

To load the tensors from the Cloud Storage file we need to create `SafetensorsStreamer` instance, perform the request, and start getting the tensors:

In [ ]:
from runai_model_streamer import SafetensorsStreamer

file_path = f"gs://{gcs_bucket}/{gcs_upload_location}"

def heavy_workload(tensor):
    # Perform heavy computation with the tensor
    # The computation is occured during the reading
    # of the rest of the tensors from the storage
    return

with SafetensorsStreamer() as streamer:
    streamer.stream_files([file_path])
    for name, tensor in streamer.get_tensors():
        heavy_workload(tensor)